# Question Data Tutorial
The goal of this tutorial is to introduce some useful functions and show how to do typical tasks when working with quantitative educational data. This tutorial assumes you already have basic knowledge of Python and Pandas and have already completed the Exam data tutorial and/or mastered the skills in that lesson.

In this lesson, you will learn the following:
* How recode variables to new values
* Change the names of data frame columns
* Wide vs long format for data frames
* Concatenate and merge data frames

***
Created by Dr. Nicholas Young

Last modified: April 3, 2025

Python version: 3.11.9

As will likely be the case for most your files, we start by importing numpy, pandas, and pyplot

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Now let's read in our question files. This time we have two files, one from a morning section of a course and one from an afternoon section of a course. To minimize the amount of typing, I'll use 'm' for morning and 'a' for afternoon in the variables I store my data.

Some background on this data:
* This instructor's exam had 4 multiple choice questions and 2 free response questions. We only have the multiple choice questions.
* 1 is correct while 0 is incorrect
* Q1 and Q4 are the same on both exams but the instructor swapped the order of Q2 and Q3 on the two exams. This means that Q2 on the morning exam is Q3 on the afternoon exam and Q2 on the morning exam is Q3 on the afternoon exam.
* The instructor also collected some data about the students taking their exam. This is stored in the `class_demographics.csv` file.

In [15]:
m_data = pd.read_csv('morning_exam.csv')
a_data = pd.read_csv('afternoon_exam.csv')

Let's start by inspecting our data and see what we are working with

In [26]:
m_data.head()

,id,section,Q1,Q2,Q3,Q4
0,100,morning,0,1,0,0
1,101,morning,0,1,1,0
2,102,morning,0,0,0,0
3,103,morning,0,1,1,0
4,104,morning,1,0,1,1


In [22]:
a_data.head()

,id,section,Q1,Q2,Q3,Q4
0,200,afternoon,0,0,B,0
1,201,afternoon,1,0,C,0
2,202,afternoon,1,1,B,0
3,203,afternoon,0,0,B,0
4,204,afternoon,0,1,D,0


You'll likely notice a problem in that Q3 is the students' responses rather than whether they are correct or not. Let's address that first.

## Changing the values of variables based on a condition.
As is often the case, our data isn't exactly the way we want it. Here, we have a column with student answers rather than whether it is correct not. We want to convert that to correct or not (1/0).

Let us assume that "B" is the correct answer to Q3 on the afternoon exam.

There are two ways to do this. Which one we want to use depends on how many different values there will be after the conversion. For a binary outcome like correct or not, we can use a logic statement. Here, we ask if Q3 is equal to B. If so, assign a 1 and otherwise, assign a zero.

In [47]:
a_data['Q3'] =(a_data['Q3'] == "B").astype(int)

Here the `as.type(int)` is important because when we test whether each response in Q3 is B, we get a list of True and False back. True is equivalent to 1 and False is equivalent to 0 so we can convert the logic variable to an integer to get ones and zeros.

If we had multiple values were assigning to (say you were trying to categorize the alternative conception that each response targets), you could use `replace`. In `replace`, you provide a dictionary with the current values followed by the new values. This provides much more flexibility than the previous way.

In [51]:
a_data['Q3'] = a_data['Q3'].replace({"A": 0, "B": 1, "C": 0, "D": 0})

If we look at the afternoon data now, we will see that all of the questions are now in correct/incorrect format

In [54]:
a_data.head()

,id,section,Q1,Q2,Q3,Q4
0,200,afternoon,0,0,1,0
1,201,afternoon,1,0,0,0
2,202,afternoon,1,1,1,0
3,203,afternoon,0,0,1,0
4,204,afternoon,0,1,0,0


## Examining performance

Now that the data are in the correct format, let's see on how the students did. We want to find the fraction of responses that are correct per question as well as the student's score.

Try this on your own in the cells below. If you are stuck, expand the header below called solution to see what I found or look back to the Exam data tutorial for useful functions.

In [77]:
# use this block to put your code for the fraction of correct responses per question
# as a hint, 55% of students answered Q2 correctly in the morning class.



In [93]:
# use this block to put your code for total score per student
# as a hint, student 101 earned a score of 2.



### Examining performance solution
Try to do this yourself before looking at the solution.

In [64]:
m_data[['Q1', 'Q2', 'Q3', 'Q4']].mean()

Q1    0.56
Q2    0.55
Q3    0.74
Q4    0.23
dtype: float64

In [62]:
a_data[['Q1', 'Q2', 'Q3', 'Q4']].mean()

Q1    0.41
Q2    0.65
Q3    0.50
Q4    0.15
dtype: float64

Because the responses are stored as correct or not, the number of correct responses is sum of the column and the number of attempts is the number of rows in the column. This is just the average so I can find the fraction of correct responses by taking the average of each column.

In [84]:
m_data['score'] = m_data['Q1'] + m_data['Q2'] + m_data['Q3'] + m_data['Q4']
a_data['score'] = a_data['Q1'] + a_data['Q2'] + a_data['Q3'] + a_data['Q4']

In [86]:
m_data.head()

,id,section,Q1,Q2,Q3,Q4,score
0,100,morning,0,1,0,0,1
1,101,morning,0,1,1,0,2
2,102,morning,0,0,0,0,0
3,103,morning,0,1,1,0,2
4,104,morning,1,0,1,1,3


Again, since each question is graded correct or not, each student's score is the number of questions they get correct, which is the sum of the four question columns.

## Combining data sets
So far, we've treated the two data sets as separate things. However, as the questions are the same, just in a different order, it might be useful to combine the data to get a better sense of how students are going on these exams. For example, we might need to collect data from multiple classes in order to get a sufficient sample size.

Let's introduce some techniques that could be useful for doing this.

If I want to combine the data together into a single data frame with 200 rows instead of second data frames with 100 rows, I need each data frame to have the same column names.

We've already met this condition so we can use the `concat` function. I include `ignore_index=True` so that a new index will be created. Otherwise, the individual indices of the existing data frames will be kept and you would have two rows with each index (0-99) because `a_data` and `m_data` each have an index of 0-99.

In [103]:
pd.concat([a_data, m_data], ignore_index=True)

,id,section,Q1,Q2,Q3,Q4,score
0,200,afternoon,0,0,1,0,1
1,201,afternoon,1,0,0,0,1
2,202,afternoon,1,1,1,0,3
3,203,afternoon,0,0,1,0,1
4,204,afternoon,0,1,0,0,1
...,...,...,...,...,...,...,...
195,195,morning,1,1,1,0,3
196,196,morning,0,0,0,1,1
197,197,morning,1,0,0,1,2
198,198,morning,1,1,1,0,3


Here's what that looks like without the `ignore_index=True` part. Notice that the first column ends at 99 now instead of 199. Generally it's a good idea to not have the same index twice.

In [106]:
pd.concat([a_data, m_data])

,id,section,Q1,Q2,Q3,Q4,score
0,200,afternoon,0,0,1,0,1
1,201,afternoon,1,0,0,0,1
2,202,afternoon,1,1,1,0,3
3,203,afternoon,0,0,1,0,1
4,204,afternoon,0,1,0,0,1
...,...,...,...,...,...,...,...
95,195,morning,1,1,1,0,3
96,196,morning,0,0,0,1,1
97,197,morning,1,0,0,1,2
98,198,morning,1,1,1,0,3


Problem with this given Q2 means different things